In [2]:
import json
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import datetime

In [4]:
def get_json(api_url):
	response = requests.get(api_url)
	if response.status_code == 200:
		return json.loads(response.content.decode('utf-8'))
	else:
		return None
    

In [5]:
record_date = '2020-08-17'
covid_url = 'https://covid19-api.org/api/status?date='+record_date
df_covid_worldwide = pd.io.json.json_normalize(get_json(covid_url))

In [5]:
df_covid_worldwide.head()

,country,last_update,cases,deaths,recovered
0,US,2020-08-16T23:27:50,5401167,170019,1833067
1,BR,2020-08-16T23:27:50,3340197,107852,2655017
2,IN,2020-08-16T23:27:50,2589682,49980,1862258
3,RU,2020-08-16T23:27:50,920719,15653,731444
4,ZA,2020-08-16T23:27:50,587345,11839,472377


In [6]:
df_covid_worldwide['last_update'] = pd.to_datetime(df_covid_worldwide['last_update'], format='%Y-%m-%d %H:%M:%S')
df_covid_worldwide['last_update'] = df_covid_worldwide['last_update'].apply(lambda x: x.date())

In [7]:
countries_url = 'https://covid19-api.org/api/countries'
df_countries = pd.io.json.json_normalize(get_json(countries_url))
df_countries = df_countries.rename(columns={'alpha2': 'country'})[['name','country']]

print(df_countries.head())

          name country
0  Afghanistan      AF
1      Albania      AL
2      Algeria      DZ
3      Andorra      AD
4       Angola      AO


In [8]:
df_covid_denormalized = pd.merge(df_covid_worldwide, df_countries, on='country')
df_covid_denormalized 

,country,last_update,cases,deaths,recovered,name
0,US,2020-08-16,5401167,170019,1833067,United States of America
1,BR,2020-08-16,3340197,107852,2655017,Brazil
2,IN,2020-08-16,2589682,49980,1862258,India
3,RU,2020-08-16,920719,15653,731444,Russian Federation
4,ZA,2020-08-16,587345,11839,472377,South Africa
...,...,...,...,...,...,...
201,GG,2020-03-17,0,0,0,Guernsey
202,GL,2020-03-19,0,0,0,Greenland
203,GU,2020-03-18,0,0,0,Guam
204,JE,2020-03-17,0,0,0,Jersey


In [9]:
#fatality ratio
df_covid_denormalized['fatality_ratio'] = df_covid_denormalized['deaths']/df_covid_denormalized['cases']
df_covid_denormalized

,country,last_update,cases,deaths,recovered,name,fatality_ratio
0,US,2020-08-16,5401167,170019,1833067,United States of America,0.031478
1,BR,2020-08-16,3340197,107852,2655017,Brazil,0.032289
2,IN,2020-08-16,2589682,49980,1862258,India,0.019300
3,RU,2020-08-16,920719,15653,731444,Russian Federation,0.017001
4,ZA,2020-08-16,587345,11839,472377,South Africa,0.020157
...,...,...,...,...,...,...,...
201,GG,2020-03-17,0,0,0,Guernsey,NaN
202,GL,2020-03-19,0,0,0,Greenland,NaN
203,GU,2020-03-18,0,0,0,Guam,NaN
204,JE,2020-03-17,0,0,0,Jersey,NaN
